In [48]:
import csv
import pandas as pd
import numpy as np
import us

In [13]:
df_2021 = pd.read_csv("PERM_Disclosure_Data_FY2021.csv", low_memory=False)
df_2022 = pd.read_csv("PERM_Disclosure_Data_FY2022_Q1.csv", low_memory=False)

In [50]:
def normalize(data):
    res = data.replace(',', '')
    res = res.replace('.', '')
    res = res.upper()
    return res

def clean_up(data):
    res = data.replace('"', '')
    res = res.replace('\'', '')
    res = res.replace(',', '')
    res = res.replace('  ', '')
    res = res.upper()
    return res

def state_to_code(state_name):
    if state_name == None:
        return state_name
    state_name = ''.join([i for i in state_name if not i.isdigit()])
    state = us.states.lookup(state_name)
    if state == None:
        return state_name
    return state.abbr

In [15]:
df_2021['EMPLOYER_NAME'] = df_2021['EMPLOYER_NAME'].apply(normalize)
df_2022['EMPLOYER_NAME'] = df_2022['EMPLOYER_NAME'].apply(normalize)
companies = pd.read_csv("companies.csv", low_memory=False)
companies

,COMPANY_ID,NAME,ADDRESS_1,ADDRESS_2,CITY,STATE,COUNTRY,ZIP_CODE
0,0,400 CAPITAL MANAGEMENT LLC,510 MADISON AVE,17TH FLOOR,NEW YORK,NY,UNITED STATES OF AMERICA,10025
1,1,ABN AMRO CLEARING CHICAGO LLC,175 W JACKSON BOULEVARD,SUITE 2050,CHICAGO,IL,UNITED STATES OF AMERICA,60604
2,2,ACHELOUS PARTNERS,800 3RD AVENUE,21ST FLOOR,NEW YORK,NY,UNITED STATES OF AMERICA,10022
3,3,AES US SERVICES LLC,4300 WILSON BLVD,NaN,ARLINGTON,VA,UNITED STATES OF AMERICA,22203
4,4,AKUNA CAPITAL LLC,333 S. WABASH AVE.,SUITE 2600,CHICAGO,IL,UNITED STATES OF AMERICA,60604
...,...,...,...,...,...,...,...,...
287,287,WEST COAST FLORIDA ENTERPRISES INC,8090 SUPPLY DRIVE,STE 100,FT. MYERS,FL,UNITED STATES OF AMERICA,33912
288,288,WILLIAM BLAIR COMPANY LLC,150 N. RIVERSIDE PLAZA,NaN,CHICAGO,IL,UNITED STATES OF AMERICA,60606
289,289,WORLDQUANT LLC,1700 EAST PUTNAM AVENUE,NaN,OLD GREENWICH,CT,UNITED STATES OF AMERICA,6870
290,290,WORLDQUANT PREDICTIVE TECHNOLOGIES,575 5TH AVE,NaN,NEW YORK CITY,NY,UNITED STATES OF AMERICA,10017


In [16]:
companies2 = pd.read_csv("companies_NSCC-MPID_fuzzymatched.csv", low_memory=False)
companies2

,COMPANY_ID,NAME,ADDRESS_1,ADDRESS_2,CITY,STATE,COUNTRY,ZIP_CODE
0,292,C6 CAPITAL SECURITIES LLC,600 Lexington Ave,32nd floor,New York,NY,UNITED STATES OF AMERICA,10022 ...
1,293,NATIONAL ALLIANCE SECURITIES CORPORATION,505 Park Ave,# 502,New York,NY,UNITED STATES OF AMERICA,10022
2,294,RIM SECURITIES LLC,400 Park Ave,NaN,New York,NY,UNITED STATES OF AMERICA,10022 ...
3,295,KMS FINANCIAL SERVICES,2001 6th Ave,#2801,Seattle,WA,UNITED STATES OF AMERICA,98121
4,296,VSR FINANCIAL SERVICES,8620 W 110TH STREET,200,OVERLAND PARK,KS,UNITED STATES OF AMERICA,66210-9651
5,297,ICP SECURITIES LLC,360 Madison Ave,NaN,New York,NY,UNITED STATES OF AMERICA,10017 ...


In [17]:
companies3 = pd.read_csv("companies_H1B_fuzzymatched.csv", low_memory=False)
companies3

,COMPANY_ID,NAME,CITY,STATE
0,298,ROSEWOOD INVESTMENT CORPORATION,NEW YORK,NY
1,299,SEMPER CAPITAL MANAGEMENT LP,NEW YORK,NY
2,300,VOYA INVESTMENT MANAGEMENT LLC,ATLANTA,GA
3,301,HQ CAPITAL PRIVATE EQUITY LLC,NEW YORK,NY
4,302,WATERFALL ASSET MANAGEMENT LLC,NEW YORK,NY
...,...,...,...,...
254,552,GATEWOOD WEALTH SOLUTIONS,ST. LOUIS,MO
255,553,B RILEY FBR INC,NEW YORK,NY
256,554,ALTISOURCE SOLUTIONS INC,PLANO,TX
257,555,PEOPLE'S UNITED ADVISORS INC,NEW YORK,NY


In [56]:
company_roles = df_2021[df_2021['EMPLOYER_NAME'].isin(companies['NAME']) | df_2021['EMPLOYER_NAME'].isin(companies2['NAME']) | df_2021['EMPLOYER_NAME'].isin(companies3['NAME'])].copy()
company_roles['WORKSITE_CITY'] = company_roles['WORKSITE_CITY'].str.upper()
company_roles['WORKSITE_STATE'] = company_roles['WORKSITE_STATE'].str.upper()
company_roles['JOB_TITLE'] = company_roles['JOB_TITLE'].str.upper()
company_roles['JOB_TITLE'] = company_roles['JOB_TITLE'].apply(clean_up)
roles = company_roles['JOB_TITLE'].unique()
roles

,CASE_NUMBER,RECEIVED_DATE,DECISION_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS_1,EMPLOYER_ADDRESS_2,EMPLOYER_CITY,EMPLOYER_STATE_PROVINCE,EMPLOYER_COUNTRY,EMPLOYER_POSTAL_CODE,...,WAGE_OFFER_FROM,WAGE_OFFER_TO,WAGE_OFFER_UNIT_OF_PAY,WORKSITE_CITY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,JOB_TITLE,MINIMUM_EDUCATION,MAJOR_FIELD_OF_STUDY,SPECIFIC_SKILLS
7,A-19278-67143,11/18/2019,10/1/2020,US BANK NATIONAL ASSOCIATION,800 NICOLLET MALL,NaN,MINNEAPOLIS,MINNESOTA,UNITED STATES OF AMERICA,55402,...,"$95,742.00",NaN,Year,GRESHAM,OR,97230,AEM ADMINISTRATOR,Bachelor's,"CS, IT or related field",Requires a Bachelors degree in Computer Scienc...
8,A-19280-67307,12/20/2019,10/1/2020,BAIN COMPANY INC,131 DARTMOUTH STREET,NaN,BOSTON,MASSACHUSETTS,UNITED STATES OF AMERICA,2116,...,"$185,000.00","$225,000.00",Year,NEW YORK,NY,10036,MANAGER,Master's,Business Administration,Experience must include valuation of product l...
18,A-19308-78622,11/15/2019,10/1/2020,TWO SIGMA INVESTMENTS LP,"100 AVENUE OF THE AMERICAS, FLOOR 16",NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10013,...,"$170,000.00",NaN,Year,NEW YORK,NY,10013,QUANTITATIVE SOFTWARE ENGINEER,Master's,Computer Science,Knowledge of the following software technologi...
19,A-19309-79480,12/23/2019,10/1/2020,CITADEL SECURITIES AMERICAS LLC,131 S DEARBORN STREET,NaN,CHICAGO,ILLINOIS,UNITED STATES OF AMERICA,60603,...,"$150,000.00",NaN,Year,CHICAGO,IL,60603,ANALYST,Bachelor's,"Math, Statistics, Computer Science, Engineerin...",Requires a Bachelors degree or foreign equival...
150,A-20062-25465,4/28/2020,10/1/2020,KPMG LLP,2323 ROSS AVENUE,SUITE 1400,DALLAS,TEXAS,UNITED STATES OF AMERICA,75201,...,"$128,000.00",NaN,Year,NEW YORK,NY,10154,SENIOR MANAGER,Bachelor's,Accounting or related field,Three 3 years of experience must include Perfo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107755,A-20216-84079,11/17/2020,9/28/2021,JP MORGAN CHASE CO,10 SOUTH DEARBORN STREET,IL10043,CHICAGO,ILLINOIS,UNITED STATES OF AMERICA,60603,...,"$100,000.00",NaN,Year,PLANO,TX,75024,ASSOCIATE SOFTWARE ENGINEERING,Bachelor's,"Computer Science, Information Systems, or rela...",Must have experience developing JavaJ2EEWeb co...
107765,A-20258-98920,11/20/2020,9/28/2021,BLOOMBERG LP,731 LEXINGTON AVE.,NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10022,...,"$145,000.00","$180,000.00",Year,NEW YORK,NY,10022,SOFTWARE ENGINEER,Master's,"Computer Science, Electrical Engineering, Info...",Must have education or experience in each of t...
107767,A-20260-00053,11/23/2020,9/28/2021,BLOOMBERG LP,731 LEXINGTON AVE.,NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10022,...,"$113,000.00","$211,000.00",Year,NEW YORK,NY,10022,CROSS ASSET FINANCIAL ENGINEER,Master's,"Math, Physics or Engineering or related",Must have education or experience in each of t...
107922,A-20260-00138,11/30/2020,9/29/2021,BNP PARIBAS SECURITIES CORP,525 Washington BLVD,NaN,Jersey City,NEW JERSEY,UNITED STATES OF AMERICA,7310,...,"$210,000.00",NaN,Year,NEW YORK,NY,10019,VICE PRESIDENT,Bachelor's,Business Administration or Finance,Bachelors degree in Business Administration or...


In [59]:
company_roles2 = df_2022[df_2022['EMPLOYER_NAME'].isin(companies['NAME']) | df_2022['EMPLOYER_NAME'].isin(companies2['NAME']) | df_2022['EMPLOYER_NAME'].isin(companies3['NAME'])].copy()
company_roles2['WORKSITE_CITY'] = company_roles2['WORKSITE_CITY'].str.upper()
company_roles2['WORKSITE_STATE'] = company_roles2['WORKSITE_STATE'].str.upper()
company_roles2['JOB_TITLE'] = company_roles2['JOB_TITLE'].str.upper()
company_roles2['JOB_TITLE'] = company_roles2['JOB_TITLE'].apply(clean_up)
roles2 = company_roles2['JOB_TITLE'].unique()
roles2

,CASE_NUMBER,RECEIVED_DATE,DECISION_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS_1,EMPLOYER_ADDRESS_2,EMPLOYER_CITY,EMPLOYER_STATE_PROVINCE,EMPLOYER_COUNTRY,EMPLOYER_POSTAL_CODE,...,WAGE_OFFER_FROM,WAGE_OFFER_TO,WAGE_OFFER_UNIT_OF_PAY,WORKSITE_CITY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,JOB_TITLE,MINIMUM_EDUCATION,MAJOR_FIELD_OF_STUDY,SPECIFIC_SKILLS
15,A-21102-73106,4/21/2021,10/1/2021,VIRTU FINANCIAL OPERATING LLC,"165 BROADWAY, 1 LIBERTY PLAZA",NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10006,...,"$72,363.00","$150,000.00",Year,NEW YORK,NY,10006,QUANTITATIVE TRADER,Bachelor's,"Computer Science, Mathematics, Computational F...",Bachelors degree or foreign equivalent in Comp...
245,A-21065-59135,5/3/2021,10/4/2021,TD SECURITIES USA LLC,1 VANDERBILT AVENUE,NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10017,...,"$133,250.00",NaN,Year,CHARLOTTE,NC,28277,DIRECTOR FINANCE,Master's,"Finance, Economics, Business Administration, o...","Position requires a Masters degree, or foreign..."
278,A-21051-54138,5/9/2021,10/4/2021,BANK OF AMERICA,100 N. TRYON STREET,NaN,CHARLOTTE,NORTH CAROLINA,UNITED STATES OF AMERICA,28255,...,"$210,000.00",NaN,Year,NEW YORK,NY,10038,DIRECTOR / BA LEAD MKTS (MGR),Bachelor's,"Bus, Fin, Appl Math, Stat or related",Must have 7 years of progressively responsible...
291,A-21029-47138,5/7/2021,10/4/2021,BLACKROCK FINANCIAL MANAGEMENT,40 EAST 52ND STREET,NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10022,...,"$95,000.00",NaN,Year,WILMINGTON,DE,19809,ASSOCIATE GLOBAL FUND AND ACCOUNTING SERVICES,Master's,"Finance, Economics, Statistics, or related",Three 3 years of experience in the job offered...
295,A-21055-55295,5/12/2021,10/4/2021,MORGAN STANLEY SERVICES GROUP INC,1585 BROADWAY,NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10036,...,"$192,000.00",NaN,Year,NEW YORK,NY,10019,EXECUTIVE DIRECTOR CREDIT RISK,Bachelor's,Computer Engineering,Requires five 5 years of experience with ident...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27325,A-21173-04961,7/15/2021,12/27/2021,MORGAN STANLEY CO LLC,1585 BROADWAY,NaN,NEW YORK,NEW YORK,UNITED STATES OF AMERICA,10036,...,"$175,000.00",NaN,Year,NEW YORK,NY,10036,ASSOCIATE STRATS,Master's,Financial Engineering,Requires any amount of experience with Calcula...
27465,A-21196-18084,7/15/2021,12/28/2021,US BANK NATIONAL ASSOCIATION,800 NICOLLET MALL,NaN,MINNEAPOLIS,MINNESOTA,UNITED STATES OF AMERICA,55402,...,"$95,037.00",NaN,Year,MINNEAPOLIS,MN,55402,CHIEF ENGINEER (MULTIPLE POSITIONS),Master's,"Computer, Eng, Tech field","Requires a masters degree in a Computer, Engin..."
27494,A-21302-65559,11/23/2021,12/28/2021,JP MORGAN CHASE CO,10 SOUTH DEARBORN STREET,IL10043,CHICAGO,ILLINOIS,UNITED STATES OF AMERICA,60603,...,"$152,000.00",NaN,Year,JERSEY CITY,NJ,7310,VICE PRESIDENT SOFTWARE ENGINEERING,Master's,"***Please see section H.14, specific skills or...",Skills required Experience with Python. Experi...
27569,A-21178-08299,7/12/2021,12/28/2021,JP MORGAN CHASE CO,10 SOUTH DEARBORN STREET,IL10043,CHICAGO,ILLINOIS,UNITED STATES OF AMERICA,60603,...,"$98,000.00",NaN,Year,COLUMBUS,OH,43219,ASSOCIATE APPLICATIONS SUPPORT,Bachelor's,"Science, Technology, Engineering, or related f...","Must have demonstrated knowledge of WebSphere,..."


In [14]:
h1b = pd.read_csv('company_roles_h1b.csv', low_memory=False)
h1b['ROLE_NAME'] = h1b['ROLE_NAME'].astype(str)
h1b['ROLE_NAME'] = h1b['ROLE_NAME'].str.upper()
h1b['ROLE_NAME'] = h1b['ROLE_NAME'].apply(clean_up)
h1b_roles = h1b['ROLE_NAME'].unique()
h1b_roles

array(['ASSOCIATE QUANTITATIVE ANALYTICS', 'INVESTMENT ANALYST',
       'ANALYST FINANCE', ..., 'HEAD OF CORE DEVELOPMENT',
       'SENIOR ACCOUNTING OPERATIONS ANALYST', 'RESEARCH ADMINISTRATOR'],
      dtype=object)

In [16]:
all_roles = np.unique(np.concatenate((roles, roles2, h1b_roles), axis=None))
all_role_df = pd.DataFrame(all_roles, columns = ['NAME'])
all_role_df = all_role_df.reset_index(drop=True)
all_role_df.index.name = 'ROLE_ID'
all_role_df.to_csv('roles.csv')

In [17]:
roles = pd.read_csv("roles.csv", low_memory=False)
roles

,ROLE_ID,NAME
0,0,.NET & FRONT-END DEVELOPER
1,1,.NET APPLICATION DEVELOPER- TEST AUTOMATION
2,2,.NET AUTOMATION DEVELOPER
3,3,.NET DEVELOPER
4,4,.NET DEVELOPER (ASSISTANT VICE PRESIDENT)
...,...,...
23623,23623,WORLD TRADER AND GLOBALIZATION SENIOR REPORTER
23624,23624,WQ INFRASTRUCTURE ALIGNED SYSTEMS ENGINEER
23625,23625,XVA QUANT- VICE PRESIDENT
23626,23626,YIELD BOOK ANALYST


In [18]:
cr = pd.DataFrame(columns = ['COMPANY_ID', 'ROLE_ID'])

for index, row in companies.iterrows():
    temp = company_roles.loc[company_roles['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr


,COMPANY_ID,ROLE_ID
0,0,2989
1,1,20979
2,2,11134
3,3,13545
4,3,15110
...,...,...
4987,289,14738
4988,289,16777
4989,289,5870
4990,290,17480


In [19]:
for index, row in companies3.iterrows():
    temp = company_roles.loc[company_roles['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr

,COMPANY_ID,ROLE_ID
0,0,2989
1,1,20979
2,2,11134
3,3,13545
4,3,15110
...,...,...
5185,547,18132
5186,547,17808
5187,547,17571
5188,552,13838


In [20]:
for index, row in companies.iterrows():
    temp = company_roles2.loc[company_roles2['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr

,COMPANY_ID,ROLE_ID
0,0,2989
1,1,20979
2,2,11134
3,3,13545
4,3,15110
...,...,...
6478,285,9620
6479,285,6816
6480,288,20660
6481,289,3776


In [20]:
for index, row in companies3.iterrows():
    temp = company_roles2.loc[company_roles2['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr

,COMPANY_ID,ROLE_ID
0,0,2931
1,1,20040
2,2,10749
3,3,13003
4,3,14523
...,...,...
5064,514,15946
5065,519,15946
5066,524,15357
5067,524,16218


In [21]:
cr.to_csv('temp_company_roles.csv')

In [21]:
h1b.drop(['ROLE_ID'], axis=1, inplace=True)
h1b.rename(columns={"ROLE_NAME": "NAME"}, inplace=True)
h1b

,COMPANY_ROLES_ID,COMPANY_ID,NAME
0,0,0,ASSOCIATE QUANTITATIVE ANALYTICS
1,1,0,ASSOCIATE QUANTITATIVE ANALYTICS
2,2,0,ASSOCIATE QUANTITATIVE ANALYTICS
3,3,0,INVESTMENT ANALYST
4,4,0,ASSOCIATE QUANTITATIVE ANALYTICS
...,...,...,...
64356,64356,556,QUANTITATIVE DATA STRATEGIST
64357,64357,556,PORTFOLIO MANAGER
64358,64358,556,RISK ANALYST
64359,64359,556,ANALYST


In [24]:
new_cr = h1b.merge(roles,on='NAME',how='left')
new_cr.drop(['COMPANY_ROLES_ID', 'NAME'], axis=1, inplace=True)
new_cr

,COMPANY_ID,ROLE_ID
0,0,3763
1,0,3763
2,0,3763
3,0,11079
4,0,3763
...,...,...
64356,556,14945
64357,556,13813
64358,556,15452
64359,556,396


In [25]:
all_comp_roles = new_cr.append(cr)
all_comp_roles = all_comp_roles.drop_duplicates()
all_comp_roles = all_comp_roles.reset_index()
all_comp_roles.drop(['index'], axis=1, inplace=True)
all_comp_roles.index.name = 'COMPANY_ROLES_ID'
all_comp_roles.to_csv('company_roles.csv')